In [29]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment

from pathlib import Path

import copy 
import pandas as pd
import datetime

from scipy.signal import find_peaks

import os

# get dictools stuff 
import sys
sys.path.append("c:/Ben/Work/hrdic-tools/")
import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

In [70]:
def newtec_rigdata_read(rig_file_name,gauge_length,gauge_width,gauge_thickness):
    # reads in the two key datafiles from Softstrain
    # 1. csv file containing the rig data
    # 2. log file containing imaging and step timings 

    # This function combines everything into one giant dataframe. Unfortunately the timestamps are different for the log and the csv so we have to do some "filling"

    # Read the csv file 
    csv_file = rig_file_name + '.csv'
    df_csv = pd.read_csv(csv_file)

    # change to work with timestamps rather than "Time"
    df_csv['Time stamp'] = pd.to_datetime(df_csv['Time'])

    # get rid of these as metadata may be wrong meaning these are wrong
    df_csv.drop(columns = ['Strain (%)','Stress (MPa)','Corrected Elongation (µm)','Corrected Strain (%)','Time'],inplace=True)


    # Read log file 
    # read log file to get everything 
    log_file = rig_file_name + '_log.txt'
    df_log = pd.read_csv(log_file,names=['Raw'])

    # separate out lines into timestamps and content 
    df_log[['Time stamp','Text']] = df_log['Raw'].str.split(pat=r"PROJECT|SEM MANAGER|TRACTION",expand=True,regex=True)
    df_log['Time stamp'] = df_log['Time stamp'].str.replace('[INFO] ','')

    df_log['Time stamp'] = pd.to_datetime(df_log['Time stamp'])

    # separate out SEM MANAGER lines 
    df_log['Type'] = None
    df_log.loc[pd.notnull(df_log['Raw'].str.split(pat=r"SEM MANAGER",expand=True,regex=True)[1]),'Type'] = 'SEM MANAGER'

    # separate out PROJECT lines
    df_log.loc[pd.notnull(df_log['Raw'].str.split(pat=r"PROJECT",expand=True,regex=True)[1]),'Type'] = 'PROJECT'

    # separate out PROJECT lines
    df_log.loc[pd.notnull(df_log['Raw'].str.split(pat=r"TRACTION",expand=True,regex=True)[1]),'Type'] = 'TRACTION'

    
    # Join the two dataframes together
    # join everything together
    df_mega = pd.concat([df_csv,df_log])

    # sort so timestamps increase monotonically
    df_mega = df_mega.sort_values('Time stamp')

    # reindex to destroy old indices from previous dataframes
    df_mega = df_mega.reset_index(drop=True)

    # tidy up some unnecessary columns
    df_mega = df_mega.drop(columns = ['Raw',
                                    'Elapsed Time (s)',
                                    'LVDT Position (µm)'])



    # fill in blanks for timestamps that don't match imaging timestamps 
    # backfill first
    df_mega['Position (µm)']            = df_mega['Position (µm)'].bfill()
    df_mega['Elongation (µm)']          = df_mega['Elongation (µm)'].bfill()
    df_mega['Force (N)']                = df_mega['Force (N)'].bfill()
    df_mega['Step']                     = df_mega['Step'].bfill()
    df_mega['Sample Temperature (°C)']  = df_mega['Sample Temperature (°C)'].bfill()

    

    # then foward fill to deal with the very end
    df_mega['Position (µm)']            = df_mega['Position (µm)'].ffill()
    df_mega['Elongation (µm)']          = df_mega['Elongation (µm)'].ffill()
    df_mega['Force (N)']                = df_mega['Force (N)'].ffill()
    df_mega['Step']                     = df_mega['Step'].ffill()
    df_mega['Sample Temperature (°C)']  = df_mega['Sample Temperature (°C)'].ffill()


    # calculate stress and strain measures 
    gauge_csa = gauge_width*gauge_thickness*1e-6 # m^2
    df_mega['Eng Stress / MPa']  = df_mega['Force (N)']*1e-6/gauge_csa
    df_mega['Eng Strain / - ']   = (df_mega['Elongation (µm)']-df_mega['Elongation (µm)'].iloc[0])*1e-3 / gauge_length

    df_mega['True Stress / MPa'] = df_mega['Eng Stress / MPa']*(1 + df_mega['Eng Strain / - '])
    df_mega['True Strain / - ']  = np.log(1+df_mega['Eng Strain / - '])

    # add one to steps as (for some unknown reason), Softstrain starts at -1
    df_mega['Step'] = df_mega['Step'] + 1

    # make a new elapsed time column
    t_delta = df_mega['Time stamp']- df_mega['Time stamp'].iloc[0]

    # convert to seconds. Pandas by default goes to nanoseconds so convert to seconds
    t_delta = 1e-9*pd.to_numeric(t_delta)
    df_mega['Elapsed Time (s)'] = t_delta


    # find imaging steps
    df_mega['Imaging step'] = False 
    df_mega['Imaging start'] = False

    for i in range(0,np.asarray(df_mega['Step'].unique(),dtype=int)[-1]+1):
        this_step = df_mega[df_mega['Step'] == i] 

        # word that triggers this is "Acquisition" - find first occurence
        try:
            # checks if step contains an "Acquisition"
            idx = this_step['Text'][this_step['Text'].str.contains('Acquisition',na=False)].index[0]

            # sets imaging step boolean to true if there is an acquisition
            df_mega['Imaging step'] = df_mega['Imaging step'].mask(df_mega['Step'] == i,True)

            # sets imaging start boolean to true for time stamp of imaging start
            df_mega.at[idx,'Imaging start'] = True

        except:
            # not an imaging step so we can ignore 
            pass






    # plot a figure to show the test profiles
    fig = plt.figure()

    fig.set_size_inches(12, 7)

    # set up grid 
    gs0 = gridspec.GridSpec(nrows=1,ncols=2,figure=fig,width_ratios=[1,3])
    gs00 =gridspec.GridSpecFromSubplotSpec(nrows=3,ncols=1,subplot_spec=gs0[0])
    gs01 =gridspec.GridSpecFromSubplotSpec(nrows=1,ncols=1,subplot_spec=gs0[1:])

    ax_small = gs00.subplots(sharex=True)

    ax_big = fig.add_subplot(gs01[0])

    for i in range(0,np.asarray(df_mega['Step'].unique(),dtype=int)[-1]+1):
        this_step = df_mega[df_mega['Step'] == i]

        #plot time as elapsed from start 

        # force - time
        ax_small[0].plot(this_step['Elapsed Time (s)']/3600,this_step['Force (N)'])

        # elongation - time
        ax_small[1].plot(this_step['Elapsed Time (s)']/3600,this_step['Elongation (µm)'])

        # Temperature - time
        ax_small[2].plot(this_step['Elapsed Time (s)']/3600,this_step['Sample Temperature (°C)'])

        # Engineering stress - engineering strain
        ax_big.plot(this_step['Eng Strain / - '],this_step['Eng Stress / MPa'])

        # plot markers for when imaging starts 
        im_start = this_step[this_step['Imaging start'] == True]
        ax_big.plot(im_start['Eng Strain / - '],im_start['Eng Stress / MPa'],'kx')

        # add annotation for step number
        try:
            x0 = im_start['Eng Strain / - '].to_numpy()[0]
            y0 = im_start['Eng Stress / MPa'].to_numpy()[0]
            dx = 0.0005
            dy = -10
            
            ax_big.annotate(str(i),xy = (x0,y0), xytext = (x0+dx,y0+dy),arrowprops=dict(width=1,facecolor='black',headwidth=10,headlength=15))
        except: 
            pass

    # label it all up
    fig.suptitle('Test profile')

    # time dependent graphs
    ax_small[-1].set_xlabel('Time / hours')

    ax_small[0].set_ylabel('Force / N')
    ax_small[1].set_ylabel('Elongation (µm)')
    ax_small[2].set_ylabel('Sample Temperature (°C)')

    # flow curve
    ax_big.set_xlabel('Engineering strain / -')
    ax_big.set_ylabel('Engineering Stress / MPa')

    # switch grids on 
    for ax in ax_small: ax.grid()
    ax_big.grid()


    plt.tight_layout()



    return df_mega

In [ ]:
gauge_length = 12.0      # mm
gauge_width = 1.5        # mm
gauge_thickness = 1.006 # mm
rig_data_file = './ofhc_cu_5pc'


df = newtec_rigdata_read('./ofhc_cu_5pc',gauge_length,gauge_width,gauge_thickness)

In [33]:
this_step = df[df['Step'] == 19]
im_start = this_step[this_step['Imaging start'] == True]




In [ ]:
exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
for dic_file in sorted(dic_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)
    # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
    print(dic_map)
    
ebsd_pre_frame = experiment.Frame()
ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

In [ ]:
dic_map = exp.increments[-1].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.2)

ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

In [ ]:
ebsd_pre_frame.homog_points

In [ ]:
dic_frame.homog_points = [(1722, 1177),
 (1335, 1313),
 (948, 1745),
 (549, 1661),
 (245, 1680),
 (233, 1300),
 (579, 1158),
 (443, 756),
 (559, 136),
 (83, 359),
 (1438, 495),
 (857, 165)]

In [ ]:
ebsd_pre_frame.homog_points = [(2541, 1823),
 (2240, 1932),
 (1940, 2266),
 (1631, 2207),
 (1394, 2225),
 (1388, 1933),
 (1659, 1820),
 (1554, 1519),
 (1648, 1054),
 (1279, 1219),
 (2323, 1319),
 (1875, 1080)]

In [ ]:
 # plot maps
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_crop
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    # dic_map.plot_map(
    #     'max_shear', vmin=0, vmax=0.10, 
    #     plot_scale_bar=False, plot_gbs='line'
    # )
    # plt.tight_layout()

In [ ]:
# max force times 
force = df['Force (N)']

df['Max force in step'] = False
df['Imaging start'] = False
df['Imaging now'] = False

for i in imaging_step_numbers:
    this_step = df[df['Step'] == i]

    # find max force from cycle
    try:
        idx = this_step['Force (N)'].idxmax()
    except: 
        print('Test failed on step ' +str(i) + ' so we will ignore it')

    # plt.figure()
    # plt.plot(this_step['Elapsed Time (s)'],this_step['Force (N)'])
    # plt.plot(this_step['Elapsed Time (s)'].iloc[0],this_step['Force (N)'].iloc[0],'+')
    # plt.plot(this_step['Elapsed Time (s)'].loc[idx],this_step['Force (N)'].loc[idx],'+')

    df.loc[idx,['Max force in step']] = True

    # find imaging start time 
    
    # find mean time increment for step
    t_step = np.nanmean(np.diff(this_step['Elapsed Time (s)']))





In [ ]:
t1 = imaging_maxforce_df['Elapsed Time (s)'].to_numpy()+imaging_wait_time # <------ this might not work given we only wait on some steps! will need the metadata from the raw images to get this
t2 = np.isclose(df['Elapsed Time (s)'],t1[0],atol=0.5) 
df[t2]

In [ ]:
plt.plot(times,'+')
plt.plot(image_start_time,'+')

In [ ]:
plt.figure()
plt.plot(imaging_starts_df['Eng Strain / - '],imaging_starts_df['Eng Stress / MPa'],'+')
plt.plot(imaging_maxforce_df['Eng Strain / - '],imaging_maxforce_df['Eng Stress / MPa'],'x')
plt.plot(df['Eng Strain / - '],df['Eng Stress / MPa'],'.')

In [ ]:
xvar = 'Elapsed Time (s)'
yvar = 'Eng Stress / MPa'

plt.figure()
plt.plot(imaging_starts_df[xvar],imaging_starts_df[yvar],'+')
plt.plot(imaging_maxforce_df[xvar],imaging_maxforce_df[yvar],'x')
plt.plot(df[xvar],df[yvar],'-')